In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir("C:/Users/Cynthia/Desktop/Bootcamp_Github/Basketball-Analysis-Repo/data")

In [3]:
# Draft data

draft = pd.read_csv("orig/1976_to_2015_Draftees.csv")
draft.head(3)

,draft_Player,draft_All_NBA,draft_All.Star,draft_Draft_Yr,draft_Pk,draft_Team,draft_College,draft_Yrs,draft_Games,draft_Minutes.Played,...,draft_Executive,draft_Tenure,draft_Exec_ID,draft_Exec_draft_exp,draft_attend_college,draft_first_year,draft_second_year,draft_third_year,draft_fourth_year,draft_fifth_year
0,Robert Parish,2,9,1976,8,GSW,Centenary College of Louisiana,21,1611,45704,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0
1,Sonny Parker,0,0,1976,17,GSW,Texas A&M University,6,452,10916,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0
2,Marshall Rogers,0,0,1976,34,GSW,University of Texas-Pan American,1,26,176,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0


In [4]:
# NBA salary and productivity

nba1 = pd.read_csv("orig/NBA sal prod.csv")

# Set Roster = 1 if they played the season
nba1["Roster"] = 1
nba1.head(3)

,nba_Year,nba_Player,nba_Production,nba_TrueSalary,nba_Height,nba_Weight,Roster
0,1978,Truck Robinson,"$3,900,000",8800000.0,79.0,225.0,1
1,1978,Randy Smith,"$16,200,000",14800000.0,75.0,180.0,1
2,1978,Elvin Hayes,"$14,200,000",14700000.0,81.0,235.0,1


In [5]:
# NBA season stats

nba2 = pd.read_csv("orig/Seasons_Stats.csv")
nba2.head(3)

,nba_Year,nba_Player,nba_Pos,nba_Age,nba_Tm,nba_G,nba_GS,nba_MP,nba_PER,nba_TS%,...,nba_FT%,nba_ORB,nba_DRB,nba_TRB,nba_AST,nba_STL,nba_BLK,nba_TOV,nba_PF,nba_PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0


In [6]:
# Match draft data with next year's NBA sal and season data

dfs = []

for y in range(2000,2016):
    draft_yr = draft.loc[draft["draft_Draft_Yr"] == y]
    nba1_yr = nba1.loc[nba1["nba_Year"] == y + 1]
    nba2_yr = nba2.loc[nba2["nba_Year"] == y + 1]
    draft_nba1_yr = pd.merge(draft_yr, nba1_yr, how = "left",
                            left_on = "draft_Player", right_on = "nba_Player")
    draft_nba2_yr = pd.merge(draft_nba1_yr, nba2_yr, how = "left",
                            on = "nba_Player")
    dfs.append(pd.DataFrame(draft_nba2_yr))

In [7]:
# Create draft/nba matched data for all years
df = dfs[0]
for i in range(1,15):
    df = df.append(dfs[i])

In [8]:
# Drop duplicate columns
df = df.drop_duplicates(["draft_Draft_Yr", "draft_Player"])

In [9]:
# Fill Roster w 0 if draft pick did not play the season
df["Roster"] = df["Roster"].fillna(0)

In [10]:
# Fill missing NBA teams
players = ["Yao Ming",
           "Peter John Ramos",
           "Louis Williams",
           "Bill Walker",
           "Luc Mbah a Moute",
           "Patrick Mills",
           "Glenn Robinson III"]
teams = ["HOU",
         "WAS",
         "PHI",
         "BOS",
         "MIL",
         "POR",
         "MIN"]

for i in range(0,7):
    df.loc[df["draft_Player"] == players[i], "nba_Tm"] = teams[i]
    print(df.loc[df["draft_Player"] == players[i]][["draft_Player","nba_Tm"]])

  draft_Player nba_Tm
7     Yao Ming    HOU
        draft_Player nba_Tm
26  Peter John Ramos    WAS
     draft_Player nba_Tm
4  Louis Williams    PHI
   draft_Player nba_Tm
16  Bill Walker    BOS
        draft_Player nba_Tm
29  Luc Mbah a Moute    MIL
     draft_Player nba_Tm
55  Patrick Mills    POR
          draft_Player nba_Tm
28  Glenn Robinson III    MIN


In [11]:
# Replace names of teams that got combined
replacements = {
    "NJN": ["BRK"],
    "VAN": ["MEM"],
    "SEA": ["OKC"]
}
df.replace(replacements, inplace = True)

In [12]:
# Traded - drafted team does not match NBA team
df.loc[df["Roster"] == 1 & (df["draft_Team"] != df["nba_Tm"])][["draft_Team","nba_Tm"]].head(10)

,draft_Team,nba_Tm
3,NOH,CHH
6,HOU,MIL
7,HOU,DAL
8,DEN,TOR
11,DAL,HOU
19,MIL,HOU
29,CHI,CLE
35,CLE,CHI
39,ORL,LAC
40,ORL,TOT


In [13]:
# ID traded players

conditions = [
    (df["Roster"] == 1 & (df["draft_Team"] != df["nba_Tm"])),
    (df["Roster"] == 0),
    (df["Roster"] == 1 & (df["draft_Team"] == df["nba_Tm"]))
]
choices = [1, None, 0]

df["Traded"] = np.select(conditions, choices)

df["Traded"].value_counts()

0    448
1    207
Name: Traded, dtype: int64

In [14]:
# Save data

df.to_csv("clean/draft_nba.csv", index = False)